# Predicción para Kaggle

> Modelo optimizado semillerio

> Semillerío con modelo Denicolay

In [1]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.

[notice] A new release of pip is available: 24.3 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Librerias
import os
import pandas as pd
import numpy as np
import datetime
import joblib

####################
# funciones y clases
from utils import psi, drift_deflacion

#######
# rutas
# datasets
from config import dataset_file_fe6_6xpqt
# optimizacion
from config import db_path
# modelos
from config import modelos_path
# predicciones
from config import pred_path

##########
# pipeline
from processing import ModelPipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

##############
# optimización
import optuna

#########
# modelos
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

import warnings

# Ignorar advertencias de tipo UserWarning
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

Matplotlib is building the font cache; this may take a moment.
/home/santtedo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Variables de train y test

In [ ]:
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train_all = [201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
                 201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
                 202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
                 202101, 202102, 202103, 202104, 202105, 202106, 202107]

mes_train = [202107]
mes_test = 202109

threshold = 0.0195

semillas = [437809, 327347, 392879, 455783, 217163]

## Loading data

In [ ]:
data = pd.read_parquet(dataset_file_fe6_6xpqt)

label_mapping = {'CONTINUA': 0, 'BAJA+1': 1, 'BAJA+2': 2}

data['clase_ternaria'] = data['clase_ternaria'].map(label_mapping)

mes_bt_train_all = [
                    # 201901, 201902, 201903, 201904, 201906, 201907, # no los agregó por rotura de variables creadas
                    201908, 201909, 201911, 201912, 202001, 202002, 
                    202003, 202008, 202009, 202010, 202011, 202012,
                    202101, 202102, 202103, 202104, 202105, 202103,
                    202107] # igualmente habra variables perturbadas en los meses que no son tenidos en cuenta

meses_train = 19
X_train = data[data['foto_mes'].isin(mes_bt_train_all)]
y_train = X_train['clase_ternaria']
X_train = X_train.drop(columns=['clase_ternaria'])

X_kaggle = data[data['foto_mes'] == mes_test]
X_kaggle = X_kaggle.drop(columns=['clase_ternaria']) # nulls

del data

Preprocesando data

sin normalizar

In [5]:
# Imputacion de Xs
cols_with_all_nan = X_train.columns[X_train.isna().all()].tolist()
print("Columns with all NaN values:", cols_with_all_nan)
X_train = X_train.drop(columns=cols_with_all_nan)
X_kaggle = X_kaggle.drop(columns=cols_with_all_nan)

# Imputación de nulls
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_imp = pd.DataFrame(imp_median.fit_transform(X_train), columns=X_train.columns)
X_kaggle_imp = pd.DataFrame(imp_median.transform(X_kaggle), columns=X_train.columns)

del X_train
del X_kaggle

# Codificar variables categóricas
categorical_features = [col for col in X_train_imp.columns if X_train_imp[col].dtype == 'object']

# Convertir variables categóricas a 'category' dtype para LightGBM
for col in categorical_features:
    X_train_imp[col] = X_train_imp[col].astype('category')
    X_kaggle_imp[col] = X_kaggle_imp[col].astype('category')

numero_de_cliente = X_kaggle_imp['numero_de_cliente'].astype(int)

Columns with all NaN values: ['payroll_slope_1_foto_mes', 'cuenta_corriente_slope_1_foto_mes', 'visa_consumo_slope_1_foto_mes', 'comisiones_mantenimiento_slope_1_foto_mes', 'comisiones_otras_slope_1_foto_mes']


## Modelo semillerio

**Enfoque conceptual en 6 meses + Lag1&2 + Delta1&2**

> comp03_prepro_6x

> comp03_fe6_6xx

Seleccionado según:

> comp03_back-testing_semxx

Umbral según:

> comp03_kaggle_api_sub

In [3]:
list(range(3))

[0, 1, 2]

In [ ]:
# # Definir el almacenamiento de Optuna
# storage_name = "sqlite:///" + db_path + "optimization_tree.db"

# # carga local
# # storage_name = "sqlite:///optimizacion/optimization_tree.db"
# study_name = f"exp_lgbm_comp03_local_v00"

# study = optuna.load_study(study_name = study_name,
#                                  storage = storage_name)

# # Mejores parámetros lgbm opt
# opt_params = study.best_trial.params
# opt_params.update({'n_jobs': -1})

semillero_params = {'n_estimators': 23,
                  'num_leaves': 32,
                  'learning_rate': 0.34,
                  'min_data_in_leaf': 711,
                  'feature_fraction': 0.25,
                  'extra_trees': False,
}

semillero_params.update({'n_jobs': -1})

# Entrenamiento 
print("Running semillerío para entrega")
print(semillero_params)

# para registrar las probabilidades
df_sem_proba = pd.DataFrame({
                            'client': numero_de_cliente.values,
                        })

j = 0
s_r = range(217163, 455783, 7*7*7*6) # 116 semillas
s_r = list(range(3)) # 3 semillas de prueba

s_total = len(list(s_r))
for s in s_r:
    # nueva instancia del modelos con semilla
    seed = s + (7+j)
    model = LGBMClassifier(**semillero_params, random_state=seed)
    # entreno
    print(f"Entrenando modelo con semilla: {seed}, {j+1} de {s_total}")
    model.fit(X_train_imp, y_train)
    # predigo proba
    y_pred_proba = model.predict_proba(X_kaggle_imp)
    # proba baja+2
    proba_baja2 = y_pred_proba[:,2]
    df_sem_proba[f'proba_s{seed}'] = proba_baja2
    j += 1

# Promediando proba de cada semilla
proba_s_columns = df_sem_proba.filter(regex='^proba_s')
proba_s_mean = proba_s_columns.mean(axis=1)

df_sem_proba['proba_sem_mean'] = proba_s_mean

# Umbral
thr_opt_sem = 0.0195 # segun comp03_kaggle_api_sub

# Segun Denicolay, el óptimo ronda los 11 mil estímulos

# Prediccion
df_sem_proba['pred'] = np.where(df_sem_proba.proba_sem_mean >= thr_opt_sem, 1, 0)

df_sem_proba.head()

Running semillerío para entrega
{'n_estimators': 464, 'max_leaves': 228, 'eta': 0.026841741174110256, 'gamma': 0.6065611085207565, 'min_child_weight': 10, 'subsample': 0.8649413237261332, 'colsample_bytree': 0.5013152719066779, 'n_jobs': -1}
Entrenando modelo con semilla: 217170, 1 de 25
Entrenando modelo con semilla: 226775, 2 de 25
Entrenando modelo con semilla: 236380, 3 de 25
Entrenando modelo con semilla: 245985, 4 de 25
Entrenando modelo con semilla: 255590, 5 de 25
Entrenando modelo con semilla: 265195, 6 de 25
Entrenando modelo con semilla: 274800, 7 de 25
Entrenando modelo con semilla: 284405, 8 de 25
Entrenando modelo con semilla: 294010, 9 de 25
Entrenando modelo con semilla: 303615, 10 de 25
Entrenando modelo con semilla: 313220, 11 de 25
Entrenando modelo con semilla: 322825, 12 de 25
Entrenando modelo con semilla: 332430, 13 de 25
Entrenando modelo con semilla: 342035, 14 de 25
Entrenando modelo con semilla: 351640, 15 de 25
Entrenando modelo con semilla: 361245, 16 de 25

,client,proba_s217170,proba_s226775,proba_s236380,proba_s245985,proba_s255590,proba_s265195,proba_s274800,proba_s284405,proba_s294010,...,proba_s380455,proba_s390060,proba_s399665,proba_s409270,proba_s418875,proba_s428480,proba_s438085,proba_s447690,proba_sem_mean,pred
0,563965491,0.000990,0.001111,0.001159,0.001290,0.001200,0.001157,0.001181,0.001000,0.001132,...,0.001150,0.001054,0.001137,0.001022,0.001257,0.001030,0.001131,0.001059,0.001150,0
1,549490342,0.001069,0.000947,0.001029,0.001047,0.000986,0.001098,0.000984,0.000971,0.000950,...,0.001293,0.001061,0.001277,0.000969,0.001132,0.001081,0.000973,0.001054,0.001036,0
2,683742283,0.000865,0.000822,0.000745,0.000805,0.000998,0.000862,0.000865,0.000831,0.000939,...,0.000785,0.000835,0.001048,0.000734,0.000749,0.000787,0.000826,0.000745,0.000839,0
3,281174976,0.000898,0.000847,0.000727,0.000994,0.001216,0.001045,0.000918,0.000898,0.001174,...,0.000909,0.000860,0.000939,0.001053,0.000962,0.000837,0.000993,0.001146,0.000958,0
4,725868971,0.000265,0.000245,0.000284,0.000251,0.000316,0.000247,0.000286,0.000235,0.000267,...,0.000261,0.000246,0.000286,0.000272,0.000232,0.000293,0.000279,0.000273,0.000263,0


Preparando entrega

In [7]:
submission = pd.DataFrame({
    'numero_de_cliente': numero_de_cliente.values,
    'Predicted': df_sem_proba['pred'].values
})

# Imprimir value counts de las predicciones
value_counts = submission['Predicted'].value_counts()
total_count = len(submission)
print("\nValue Counts:")
print(value_counts)
print("\nFrecuencia Relativa:")
print((value_counts / total_count) * 100)

submission.info()


Value Counts:
Predicted
0    152806
1     12838
Name: count, dtype: int64

Frecuencia Relativa:
Predicted
0    92.249644
1     7.750356
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165644 entries, 0 to 165643
Data columns (total 2 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   numero_de_cliente  165644 non-null  int64
 1   Predicted          165644 non-null  int64
dtypes: int64(2)
memory usage: 2.5 MB


Guardando semillerio opt datos x

In [ ]:
ft = "%dT-%m-%Y%H-%M-%S"
t_now = datetime.datetime.now().strftime(ft)

pred_name = f"pred_sem_03_tr{meses_train}_sem{s_total}_"+t_now+".csv"

proba_file = pred_path + "probas/" + pred_name
pred_file = pred_path + pred_name

# Guardamos las probas
df_sem_proba.to_csv(proba_file, index=False)
print(f"Probas guardadas en {proba_file}")

# Guardar el DataFrame en un archivo CSV
submission.to_csv(pred_file, index=False)
print(f"Predicciones guardadas en {pred_file}")


Probas guardadas en /home/santtedo/buckets/b1/predicciones/probas/pred_lgbm_03_tr12_sem25_01T-12-202413-28-56.csv
Predicciones guardadas en /home/santtedo/buckets/b1/predicciones/pred_lgbm_03_tr12_sem25_01T-12-202413-28-56.csv
